In [ ]:
import pypsa
import pandas as pd
import numpy as np

In [ ]:
solver = "cbc"

*** 
Check-point 1:

**In this exercise, we will import the copper plate system from Exercise 1 and incorporate a storage power plant into the network. If the system does not activate the storage component, we will explore various solutions to enforce storage behavior within the network.**

> **Remarks:** 
> 
> - While the exercises focus on electricity storage, the same concepts can be applied to create storage solutions for other energy carriers. 
> - In these exercises, we will only work with `StorageUnit` component, meaning energy-to-power ratio for storage plant is fixed. To optimise the storage energy capacity independently from the storage power capacity (e.g. in case of hydrogen or gas storage, etc), you should use a fundamental `Store` component in combination with `Link` component (Have a look at this [PyPSA example](https://pypsa.readthedocs.io/en/latest/examples/replace-generator-storage-units-with-store.html)). We will touch on this implementation on day 2 of the training.

#### Initialize network

In [ ]:
# import checkpoint point 3 network from exercise 1
n = pypsa.Network("../results/network_d1_e1-3.nc")

In [ ]:
n.generators

In [ ]:
n.loads

#### Increase `electricity_load` load to `200MW` 

In [ ]:
# You can modify network's component values directly via component's dataframe, make sure to locate correct index 
load_id = n.loads[n.loads.bus == 'electricity'].index
n.loads.loc[load_id, 'p_set'] = 200

In [ ]:
n.loads

Add a pumped-hydro power plant to the network with a fixed energy-to-power ratio of `8 hours`. The rated capacity of the plant should be endogenously decided by the model. All other techno-economic parameters are provided.

> **Source:** all costs for the example are taken from Danish energy agency technology database for energy storage (https://ens.dk/en/our-services/technology-catalogues/technology-data-energy-storage)

In [ ]:
# We need to calculate annualized capital expenditure
def calculate_annualised_capex(capex: float, interest: float, lifetime: int):
    crf = (
        interest * (1 + interest) ** lifetime / ((1 + interest) ** lifetime - 1)
    )  # Capital recovery factor
    return capex * crf

In [ ]:
# Pumped-hydro power plant techno-economic parameters are given as:
lifetime = 50
interest = 0.05
CAPEX = 600000 # $/MW
FOM = 12000  # $/MW fixed
VOM = 3.9 # $/MWh variable
fuel_cost = 0 # $/MWhth per unit water consumed
efficiency_store = 0.8 # assuming similar storing and discharging effciencies
efficiency_dispatch = 0.8 # assuming similar storing and discharging effciencies

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

n.add(
    class_name="StorageUnit",
    name="pumped_hydro_storage",
    bus="electricity",
    marginal_cost=VOM + fuel_cost,
    capital_cost=annualized_capex + FOM,
    p_nom_extendable=True,
    efficiency_store=0.8,
    efficiency_dispatch=0.8,
    p_max_pu=1,  # Discharging availability
    p_min_pu=-1,  # Charging availability
    max_hours=8,  # energy to power ratio
)

Now try to solve the network

In [ ]:
# Solve network using cbc solver
n.optimize(solver_name=solver)

Check if the storage plant is being invested or not?

In [ ]:
n.storage_units["p_nom_opt"]

Check capacities from other technologies

In [ ]:
n.generators['p_nom_opt']

Ending of check-point 1 - export network

In [ ]:
# Export network
n.export_to_netcdf("../results/network_d1_e2-1.nc")

*** 
Check-point 2:

**How to force investment into storage?**
#### TASK: Modify the network to make it invest in either of storage options

> **Hint: None of the two storage options are invested because storage is not as cost optimal as continue using solar and gas.** 

Option 1: Making cost of storage options to as lower as the model invest in instead of use other generators

In [ ]:
# import checkpoint point 1 network
n = pypsa.Network("../results/network_d1_e2-1.nc")

In [ ]:
n.storage_units.loc["pumped_hydro_storage"]

In [ ]:
# Remove capital costs of storage plants
n.storage_units.loc['pumped_hydro_storage', 'capital_cost'] = 0

In [ ]:
# Solve network again
n.optimize(solver_name=solver)

Now inspect the invested capacity and dispatch pattern of the system again.

In [ ]:
# Inspect capacity of pumped hydro storage
n.storage_units.loc['pumped_hydro_storage', 'p_nom_opt']

In [ ]:
# Inspect capacity of other plants
n.generators['p_nom_opt']

In [ ]:
# Inspect storage interaction with other powerplants to supply loads
load = n.loads_t.p 
pow_gen = n.generators_t.p
storage = n.storage_units_t.p
result = pd.concat([pow_gen, storage, load], axis=1)
result.round().head(24)

In [ ]:
plot = result.loc[:,~result.columns.isin(['electricity_load', 'nuclear_power_plant'])] # drop load and nuclear columns
plot.iloc[:48,:].plot(kind='bar', stacked=True)

Option 2: Having some initial filling for storage

> **Remarks:You can use `state_of_charge_initial` to set initial filling of an storage.**

In [ ]:
# import checkpoint point 1 network
n = pypsa.Network("../results/network_d1_e2-1.nc")

In [ ]:
# adding inital filling of storage to cover first 8 hours without sun
n.storage_units.loc['pumped_hydro_storage', 'state_of_charge_initial'] = 200*8 # load * 8hours

In [ ]:
# Solve network again
n.optimize(solver_name=solver)

Now inspect the invested capacity and dispatch pattern of the system again.

In [ ]:
n.storage_units['p_nom_opt']

In [ ]:
# Inspect capacity of other plants
n.generators['p_nom_opt']

In [ ]:
# Inspect storage interaction with other powerplants to supply loads
load = n.loads_t.p 
pow_gen = n.generators_t.p
storage = n.storage_units_t.p
result = pd.concat([pow_gen, storage, load], axis=1)

plot = result.loc[:,~result.columns.isin(['electricity_load', 'nuclear_power_plant'])] # drop load and nuclear columns
plot.iloc[:48,:].plot(kind='bar', stacked=True)